In [1]:
import random as rd

In [2]:
rd.seed(23)

In [3]:
num_date=250

In [4]:
S=2000

In [5]:
mu=0.3

In [6]:
sigma=0.7

In [7]:
delta_t=1/250

In [8]:
S_open=[]

In [9]:
S_open.append(S)

In [10]:
for i in range(2,num_date+1):
    S=round(S+mu*S*delta_t+sigma*S*rd.normalvariate(0,delta_t**0.5),2)
    S_open.append(S)
print(S_open[0:4])

[2000, 2026.66, 2034.42, 1951.18]


In [11]:
import numpy as np
#产生收盘价
S_close=S_open*(1+(np.random.rand(num_date)-0.5)*0.2)
S_close=[round(i,2) for i in S_close]
##产生最低价
d=np.c_[S_open,S_close]
#最低值是较小的值随机下浮（最多下浮10%）
S_low=[min(i) for i in d]*(np.random.rand(num_date)*(-0.1)+1)
S_low=[round(i,2) for i in S_low]
#最高价是较大值的随机上浮（最多上浮10%）
S_high=[max(i) for i in d]*(np.random.rand(num_date)*(0.1)+1)
S_high=[round(i,2) for i in S_high]
d=np.c_[d,S_low,S_high]
print((d[0:4]))

[[2000.   1927.27 1759.87 2038.  ]
 [2026.66 2227.41 2000.56 2440.96]
 [2034.42 2033.99 2027.87 2090.24]
 [1951.18 2067.18 1859.4  2138.39]]


In [12]:
from pyecharts import Kline

kline=Kline("虚拟K线图")
kline.add("日K线",["2015/5/{}".format(i+1) for i in range(31)],d[0:31])
kline.render()#文件用浏览器打开

ERROR:lml.utils:failed to import pyecharts_snapshot
Traceback (most recent call last):
  File "d:\Users\Administrator\Anaconda3\lib\site-packages\lml\utils.py", line 43, in do_import
    plugin_module = __import__(plugin_module_name)
ModuleNotFoundError: No module named 'pyecharts_snapshot'


In [13]:
import datetime

In [14]:
def create_date(datestart=None,num_date=None):
    #strptime可以将字符串转为时间格式
    datestart=datetime.datetime.strptime(datestart,'%Y-%m-%d')
    date_list=[]
    #strftime可以将时间格式转为字符串
    date_list.append(datestart.strftime('%Y-%m-%d'))
    i=0 #开始while循环
    while i<(num_date-1):
        #日期向后加一天
        datestart+=datetime.timedelta(days=+1)
        #新的日期加到原有的date_list后面
        date_list.append(datestart.strftime('%Y-%m-%d'))
        i=i+1
    return(date_list)
if __name__== '__main__':
    # 日期天数为num_date
    date_list=create_date("2015-05-01",num_date)
    print(date_list[0:4])

['2015-05-01', '2015-05-02', '2015-05-03', '2015-05-04']


In [17]:
#绘制num_date天数的K线图
kline=Kline("虚拟K线图")
kline.add("日K线",date_list,d)
kline.render(r"d:/python/K2.html")

In [19]:
kline=Kline("虚拟K线图")
kline.add(
    "日K线",
    date_list,
    d,
    mark_point=["max","min"],
    is_datazoom_show=True, #增加可以调节窗口
    #datazoom_orient="vertical"
)
kline.render(r"d:/python/K3.html")

In [20]:
import pandas as pd
#产生dataframe变量stock_data
stock_data=pd.DataFrame(d,columns=['open','close','low','high'])
stock_data['date']=date_list
#调整变量顺序
stock_data=stock_data.reindex(columns=['date','open','close','low','high'])
stock_data['volumn']=[round(i,2) for i in (np.random.rand(num_date)*2000+1000)]

#这里只要计算两个移动平均
ma_list=[5,10]
for ma in ma_list:
    stock_data['MA_'+str(ma)]=[round(i,2) for i in pd.Series(stock_data['close']).rolling(ma).mean()]
    
print(stock_data[0:10])

         date     open    close      low     high   volumn     MA_5    MA_10
0  2015-05-01  2000.00  1927.27  1759.87  2038.00  2992.41      NaN      NaN
1  2015-05-02  2026.66  2227.41  2000.56  2440.96  2750.08      NaN      NaN
2  2015-05-03  2034.42  2033.99  2027.87  2090.24  2802.32      NaN      NaN
3  2015-05-04  1951.18  2067.18  1859.40  2138.39  2772.18      NaN      NaN
4  2015-05-05  1877.66  2004.92  1748.70  2069.82  1699.27  2052.15      NaN
5  2015-05-06  1931.66  1777.14  1719.30  2011.24  2081.70  2022.13      NaN
6  2015-05-07  1940.95  2008.12  1791.89  2070.35  2531.69  1978.27      NaN
7  2015-05-08  1921.22  1881.86  1861.85  2081.14  1424.59  1947.84      NaN
8  2015-05-09  2024.06  1883.64  1714.19  2216.92  2725.92  1911.14      NaN
9  2015-05-10  2090.54  2161.34  1894.48  2361.03  2687.12  1942.42  1997.29


In [21]:
#提取日期数据
date = stock_data["date"].apply(lambda x: str(x)).tolist()
print(date[0:4])

['2015-05-01', '2015-05-02', '2015-05-03', '2015-05-04']


In [22]:
#提取k线数据，axis=1表示按列
Kdata=stock_data.apply(lambda y:[y['open'],y['close'],y['low'],y['high']],axis=1).tolist()
print(Kdata[0:4])

[[2000.0, 1927.27, 1759.87, 2038.0], [2026.66, 2227.41, 2000.56, 2440.96], [2034.42, 2033.99, 2027.87, 2090.24], [1951.18, 2067.18, 1859.4, 2138.39]]


In [25]:
from pyecharts import Line
from pyecharts import Bar
from pyecharts import Overlap
# from pyecharts import Bar,Line,Overlap
#K线图
kline=Kline()
kline.add("综合K线图",date,Kdata,is_datazoom_show=True,)
#移动平均线    
MA_lines = Line()
MA_name_list=['MA_5','MA_10']
for MA_name in MA_name_list:
    MA_lines.add(MA_name, date, stock_data[MA_name].tolist())
#成交量柱状图
vol_bar = Bar()
#tooltip_tragger指提示框触发位置，tooltip_tragger="axis" 按照轴位置
vol_bar.add("volume", date, stock_data["volume"],
             tooltip_tragger="axis", is_legend_show=False, is_yaxis_show=False, yaxis_max=7*max(stock_data["volume"]))

Final_kline= Overlap()
Final_kline.add(kline)
Final_kline.add(MA_lines)
Final_kline.add(vol_bar,yaxis_index=1, is_add_yaxis=True)
    #overlap.add(es)
Final_kline.render(r"d:/python/K4.html")#文件用浏览器打开

KeyError: 'volume'